# CUDA Kernel

This notebook provides a comprehensive guide to understanding CUDA kernels using CuPy. We will explore every component of a CUDA kernel with detailed explanations and practical examples.

In [ ]:
import cupy as cp
import numpy as np
import time

In [ ]:
# check cuda availability
print(f"CUDA available: {cp.cuda.is_available()}")
print(f"CuPy version: {cp.__version__}")

## Basic CUDA Kernel Structure

Let's start with the simplest possible kernel and explain every component.

In [ ]:
# define our first cuda kernel as a string
simple_kernel_code = r'''
// the extern "C" declaration prevents C++ name mangling
// this ensures the function name remains unchanged for CuPy to find it
extern "C" {

// __global__ qualifier indicates this function runs on GPU
// and can be called from CPU (host)
__global__ void add_constant(
    // pointer to input array in GPU memory
    float* input,
    // pointer to output array in GPU memory  
    float* output,
    // value to add (passed by value, not pointer)
    float constant,
    // total number of elements to process
    int size
) {
    // calculate unique thread ID across all blocks
    // threadIdx.x: thread index within block (0 to blockDim.x-1)
    // blockIdx.x: block index in grid (0 to gridDim.x-1)
    // blockDim.x: number of threads per block
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    
    // bounds checking - essential to avoid memory errors
    // some threads may have idx >= size due to grid/block sizing
    if (idx < size) {
        // each thread processes one element
        output[idx] = input[idx] + constant;
    }
}

}
'''

# compile the kernel
add_constant_kernel = cp.RawKernel(simple_kernel_code, "add_constant")

# demonstrate usage
# data types are important
size = cp.int32(1000)
input_array = cp.arange(size, dtype=cp.float32)
output_array = cp.zeros(size, dtype=cp.float32)
constant_value = cp.float32(10.0)

# calculate grid and block dimensions
threads_per_block = 256  # common choice, multiple of 32 (warp size)
blocks_per_grid = (size + threads_per_block - 1) // threads_per_block

print(f"Threads per block: {threads_per_block}")
print(f"Blocks per grid: {blocks_per_grid}")
print(f"Total threads: {blocks_per_grid * threads_per_block}")

# launch kernel
# syntax: kernel((grid,), (block,), (args,))
add_constant_kernel(
    (blocks_per_grid,),     # grid dimensions (number of blocks)
    (threads_per_block,),   # block dimensions (threads per block)
    (input_array, output_array, constant_value, size)  # kernel arguments
)

# verify results
expected = input_array + constant_value
print(f"\nResults match: {cp.allclose(output_array, expected)}")
print(f"Sample output: {output_array[:10]}")

## Device Functions
Device functions are helper functions that run on GPU but can only be called from other GPU functions.

In [ ]:
# kernel with device function
device_function_code = r'''
extern "C" {

// __device__ qualifier means this function:
//   - runs on GPU
//   - can only be called from GPU, the device (not from CPU, the host)
// this is useful for code reuse within kernels
__device__ float clip_value(float value, float min_val, float max_val) {
    // ensure value is within [min_val, max_val]
    if (value <= min_val) return min_val;
    if (value >= max_val) return max_val;
    return value;
}

__device__ float safe_divide(float numerator, float denominator) {
    // avoid division by zero
    if (denominator == 0.0f) return 0.0f;
    return numerator / denominator;
}

__global__ void normalise_and_clip(
    float* input,
    float* output,
    float min_val,
    float max_val,
    float scale,
    int size
) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    
    if (idx < size) {
        // use device functions for clarity and reusability
        float normalised = safe_divide(input[idx], scale);
        output[idx] = clip_value(normalised, min_val, max_val);
    }
}

}
'''

# compile and test
normalise_kernel = cp.RawKernel(device_function_code, "normalise_and_clip")

# create test data
size = cp.int32(1000)
input_data = cp.random.uniform(-100, 100, size).astype(cp.float32)
output_data = cp.zeros(size, dtype=cp.float32)

# parameters
min_clip = cp.float32(-1.0)
max_clip = cp.float32(1.0)
scale_factor = cp.float32(50.0)

# launch kernel
threads = 256
blocks = (size + threads - 1) // threads

normalise_kernel(
    (blocks,), (threads,),
    (input_data, output_data, min_clip, max_clip, scale_factor, size)
)

print(f"Input range: [{input_data.min():.2f}, {input_data.max():.2f}]")
print(f"Output range: [{output_data.min():.2f}, {output_data.max():.2f}]")
print(f"All outputs in [{min_clip}, {max_clip}]: ",
      f"{cp.all((output_data >= min_clip) & (output_data <= max_clip))}")

## 2D Thread Indexing for Image Processing

For 2D data like images, we use 2D thread indexing.

In [ ]:
# 2D kernel for image brightness adjustment
image_kernel_code = r'''
extern "C" {

__global__ void adjust_brightness_2d(
    float* image,      // input image
    float* output,     // output image
    float brightness,  // brightness adjustment
    int width,         // image width
    int height         // image height
) {
    // calculate 2D thread indices
    // for x: threadIdx.x is position in block, blockIdx.x is block number
    int x = threadIdx.x + blockIdx.x * blockDim.x;
    // for y: threadIdx.y is position in block, blockIdx.y is block number
    int y = threadIdx.y + blockIdx.y * blockDim.y;
    
    // bounds checking for 2D
    if (x < width && y < height) {
        // convert 2D coordinates to 1D array index
        // row-major order: index = y * width + x
        int idx = y * width + x;
        
        // apply brightness adjustment
        float new_value = image[idx] + brightness;
        
        // clip to valid range [0, 255] for 8-bit images
        if (new_value < 0.0f) new_value = 0.0f;
        if (new_value > 255.0f) new_value = 255.0f;
        
        output[idx] = new_value;
    }
}

}
'''

# compile kernel
brightness_kernel = cp.RawKernel(image_kernel_code, "adjust_brightness_2d")

# create test image
width, height = cp.int32(512), cp.int32(512)
test_image = cp.random.uniform(0, 200, (height, width)).astype(cp.float32)
output_image = cp.zeros_like(test_image)

# 2D thread block configuration
# common choice: 16x16 = 256 threads per block
threads_x, threads_y = 16, 16
blocks_x = (width + threads_x - 1) // threads_x
blocks_y = (height + threads_y - 1) // threads_y

print(f"Image size: {width}x{height}")
print(f"Thread block: {threads_x}x{threads_y}")
print(f"Grid size: {blocks_x}x{blocks_y}")

# launch 2D kernel
brightness_adjustment = cp.float32(50.0)
brightness_kernel(
    (blocks_x, blocks_y),    # 2D grid dimensions
    (threads_x, threads_y),  # 2D block dimensions
    (test_image, output_image, brightness_adjustment, width, height)
)

print(f"\nOriginal brightness: {test_image.mean():.2f}")
print(f"Adjusted brightness: {output_image.mean():.2f}")
print(f"Difference: {output_image.mean() - test_image.mean():.2f}")

## Shared Memory

Shared memory is fast on-chip memory shared between threads in a block.

In [ ]:
# kernel using shared memory for stencil operations
mean_filter_code = r'''
extern "C" {

__global__ void mean_filter_1d_shared(
    float* input,
    float* output,
    int size,
    int radius  // filter radius (window = 2*radius + 1)
) {
    // declare shared memory
    // __shared__ memory is shared between all threads in a block
    // much faster than global memory but limited in size
    // size must be known at compile time or allocated dynamically
    extern __shared__ float shared_data[];
    
    int tid = threadIdx.x;  // thread index within block
    int gid = threadIdx.x + blockIdx.x * blockDim.x;  // global index
    
    // each thread loads one element to shared memory
    // handle boundary conditions for first and last blocks
    if (gid < size) {
        shared_data[tid + radius] = input[gid];
    } else {
        // pad with zero for threads beyond array bounds
        shared_data[tid + radius] = 0.0f;
    }

    // load left halo elements (elements outside block needed for filter)
    if (tid < radius) {
        int left_idx = gid - radius;
        if (left_idx >= 0) {
            shared_data[tid] = input[left_idx];
        } else {
            // mirror boundary condition
            shared_data[tid] = input[-left_idx];
        }
    }

    // load right halo elements
    if (tid >= blockDim.x - radius) {
        int halo_offset = tid - (blockDim.x - radius);  // 0 to radius-1
        int right_idx = blockIdx.x * blockDim.x + blockDim.x + halo_offset;
        int shared_idx = blockDim.x + radius + halo_offset;
        
        if (right_idx < size) {
            shared_data[shared_idx] = input[right_idx];
        } else {
            // outside the block
            int overflow = right_idx - size + 1;
            int mirror_idx = size - 1 - overflow;
            if (mirror_idx < 0) mirror_idx = 0;  // ensure valid range
            shared_data[shared_idx] = input[mirror_idx];
        }
    }
    
    // synchronise threads to ensure all data is loaded
    // all threads in block must reach this point before continuing
    __syncthreads();
    
    // compute mean using shared memory
    if (gid < size) {
        float sum = 0.0f;
        int count = 0;
        
        for (int i = -radius; i <= radius; ++i) {
            int shared_idx = tid + radius + i;
            // ensure we stay within shared memory bounds
            if (shared_idx >= 0 && 
                shared_idx < blockDim.x + 2 * radius) {
                sum += shared_data[shared_idx];
                count++;
            }
        }
        
        output[gid] = sum / count;
    }
}

}
'''

# compile kernel
mean_filter_kernel = cp.RawKernel(mean_filter_code, "mean_filter_1d_shared")

# test the kernel
size = cp.int32(10000)
radius = cp.int32(3)
input_signal = cp.random.normal(0, 1, size).astype(cp.float32)
# add some noise
input_signal += cp.random.normal(0, 0.5, size).astype(cp.float32)
output_signal = cp.zeros_like(input_signal)

# configure kernel launch
threads = 256
blocks = (size + threads - 1) // threads
# calculate shared memory size needed
shared_mem_size = (threads + 2 * radius) * 4  # 4 bytes per float

print(f"Array size: {size}")
print(f"Thread block: {threads}")
print(f"Grid size: {blocks}")
print(f"Filter radius: {radius} (window size: {2*radius + 1})")
print(f"Shared memory per block: {shared_mem_size} bytes")

# launch kernel with dynamic shared memory allocation
mean_filter_kernel(
    (blocks,), (threads,),
    (input_signal, output_signal, size, radius),
    shared_mem=shared_mem_size  # specify shared memory size
)

# compare noise levels
input_std = cp.std(input_signal)
output_std = cp.std(output_signal)
print(f"\nInput signal std dev: {input_std:.4f}")
print(f"Filtered signal std dev: {output_std:.4f}")
print(f"Noise reduction: {(1 - output_std/input_std) * 100:.1f}%")
print(f"\nSample input: {input_signal[:10]}")
print(f"Sample output: {output_signal[:10]}")

## Performance Measurement
Let see if there is any performance gain using shared memory by comparing with a naive version of 1D mean filter.

In [ ]:
# naive version for 1D mean filter
navie_code = r'''
extern "C" {

__global__ void mean_filter_1d_naive(
    float* input,
    float* output,
    int size,
    int radius
) {
    // global thread index
    int gid = threadIdx.x + blockIdx.x * blockDim.x;

    float sum = 0.0f;
    int count = 0;
    
    // loop through the window [-radius, radius]
    for (int i = -radius; i <= radius; ++i) {
        int idx = gid + i;
        
        // Handle boundaries consistently with shared memory version
        if (idx < 0) {
            // mirror at left boundary
            idx = -idx;
            sum += input[idx];
            count++;
        } else if (idx >= size) {
            // for indices beyond size, the shared memory version pads with 0
            // so we should add 0 (or simply skip)
            // but still increment the count for averaging
            count++;
        } else {
            // valid index
            sum += input[idx];
            count++;
        }
    }
    
    // only write output for valid indices
    if (gid < size && count > 0) {
        output[gid] = sum / count;
    }
}

}
'''

# compile kernels
naive_kernel = cp.RawKernel(navie_code, "mean_filter_1d_naive")

# create test data
sizes = [cp.int32(1e4), cp.int32(1e5), cp.int32(1e6), cp.int32(1e7)]
radius = 8

print(f"1D Mean Filter Performance Comparison")
print(f"Filter radius: {radius} (window size: {2*radius + 1})")
print(f"{'Size':>10} | {'Naive (ms)':>11} | {'Shared (ms)':>12} | {'Speedup':>8} | {'Match':>6}")
print("-" * 60)

for size in sizes:
    # create input data
    input_data = cp.random.normal(100, 10, size).astype(cp.float32)
    output_naive = cp.zeros(size, dtype=cp.float32)
    output_shared = cp.zeros(size, dtype=cp.float32)
    
    # kernel configuration
    threads = 256
    blocks = (size + threads - 1) // threads
    shared_mem_size = (threads + 2 * radius) * 4  # bytes
    
    # warm up
    naive_kernel((blocks,), (threads,), 
                 (input_data, output_naive, size, radius))
    mean_filter_kernel((blocks,), (threads,), 
                       (input_data, output_shared, size, radius), 
                       shared_mem=shared_mem_size)
    cp.cuda.Stream.null.synchronize()
    
    # benchmark naive version
    iterations = 1000
    start = time.perf_counter()
    for _ in range(iterations):
        naive_kernel((blocks,), (threads,), 
                     (input_data, output_naive, size, radius))
    cp.cuda.Stream.null.synchronize()
    time_naive = (time.perf_counter() - start) / iterations * 1000  # ms
    
    # benchmark shared memory version
    start = time.perf_counter()
    for _ in range(iterations):
        mean_filter_kernel((blocks,), (threads,), 
                           (input_data, output_shared, size, radius), 
                           shared_mem=shared_mem_size)
    cp.cuda.Stream.null.synchronize()
    time_shared = (time.perf_counter() - start) / iterations * 1000  # ms
    
    # verify correctness
    match = "True" if cp.allclose(output_naive, output_shared, rtol=1e-5) else "False"
    speedup = time_naive / time_shared
    print(f"{size:10d} | {time_naive:11.3f} | {time_shared:12.3f} | {speedup:8.2f}x | {match:>6}")

print(f"\nWindow size: {2*radius + 1} elements")
print(f"Naive approach: each element loaded {2*radius + 1} times")
print(f"Shared approach: each element loaded once (plus halo)")
print(f"Theoretical speedup upper bound: ~{2*radius + 1}x")

## Reduction Operations
Reduction operations combine multiple values into one (e.g. sum, max, min).

In [ ]:
# parallel reduction kernels
reduction_code = r'''
extern "C" {

__global__ void sum_reduction(
    float* input,
    float* output,  // partial sums from each block
    int size
) {
    // shared memory for this block's partial sum
    extern __shared__ float sdata[];
    
    int tid = threadIdx.x;
    int gid = threadIdx.x + blockIdx.x * blockDim.x;
    
    // load data from global to shared memory
    // each thread loads one element
    if (gid < size) {
        sdata[tid] = input[gid];
    } else {
        // padding for out-of-bounds
        sdata[tid] = 0.0f;  
    }
    
    // wait for all threads to load data
    __syncthreads();
    
    // tree-based parallel reduction in shared memory
    // each iteration halves the number of active threads
    // the current implementation only works with power-of-2 blockDim.x
    for (int stride = blockDim.x / 2; stride > 0; stride /= 2) {
        if (tid < stride) {
            // thread adds element that is stride positions away
            sdata[tid] += sdata[tid + stride];
        }
        // ensure all threads have completed addition
        __syncthreads();
    }
    
    // thread 0 writes this block's result to global memory
    if (tid == 0) {
        output[blockIdx.x] = sdata[0];
    }
}

// kernel to sum the partial results
__global__ void final_sum(
    float* partial_sums,
    float* total,
    int num_blocks
) {
    // simple sequential sum for small number of blocks
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        float sum = 0.0f;
        for (int i = 0; i < num_blocks; i++) {
            sum += partial_sums[i];
        }
        *total = sum;
    }
}

}
'''

# compile kernels
reduction_kernel = cp.RawKernel(reduction_code, "sum_reduction")
final_sum_kernel = cp.RawKernel(reduction_code, "final_sum")

# test reduction
size = cp.int32(1e6)
test_data = cp.ones(size, dtype=cp.float32)  # sum should equal size

# configure kernel
threads = 256
blocks = (size + threads - 1) // threads
partial_sums = cp.zeros(blocks, dtype=cp.float32)
total_sum = cp.zeros(1, dtype=cp.float32)

print(f"Data size: {size}")
print(f"Number of blocks: {blocks}")

# first reduction pass
shared_mem_size = threads * 4  # 4 bytes per float
reduction_kernel(
    (blocks,), (threads,),
    (test_data, partial_sums, size),
    shared_mem=shared_mem_size
)

# final sum of partial results
final_sum_kernel(
    (1,), (1,),
    (partial_sums, total_sum, cp.int32(blocks))
)

print(f"\nComputed sum: {total_sum[0]}")
print(f"Expected sum: {size}")
print(f"Error: {abs(total_sum[0] - size)}")